# CC3039 — Laboratorio 6 · **Práctica Parte 2**  
**Jacobiano y Autovalores en el modelo SIR con dinámica vital**

> Este notebook calcula el Jacobiano simbólico del sistema, evalúa autovalores en el **ELE (S=N, I=0)** y en el **equilibrio endémico** (numérico), e imprime los resultados solicitados en el enunciado.



## 1) Modelo y parámetros

Con \(\beta\) **normalizada por** \(N\), las ecuaciones en el plano \((S, I)\) son:
\[
\dot S = \mu N - \beta S I - \mu S,\quad
\dot I = \beta S I - (\gamma + \mu) I.
\]

Parámetros de la **Parte 1** (R₀ > 1):


In [1]:

N = 1000
beta = 0.0005 
gamma = 0.1
mu = 0.02

R0 = (beta * N) / (gamma + mu)
print(f"N = {N}, beta = {beta}, gamma = {gamma}, mu = {mu}")
print(f"R0 = {R0:.6f}  ->  {'> 1' if R0>1 else '<= 1'}")


N = 1000, beta = 0.0005, gamma = 0.1, mu = 0.02
R0 = 4.166667  ->  > 1



## 2) Jacobiano simbólico con `sympy`



In [2]:
import sympy as sp

# Variables
S, I = sp.symbols('S I', real=True)
beta_s, gamma_s, mu_s, N_s = sp.symbols('beta gamma mu N', positive=True, real=True)

# Ecuaciones
dS = mu_s*N_s - beta_s*S*I - mu_s*S
dI = beta_s*S*I - (gamma_s + mu_s)*I

# Jacobiano simbólico
F = sp.Matrix([dS, dI])
vars_vec = sp.Matrix([S, I])
J = F.jacobian(vars_vec)

print("Jacobiano simbólico:")
sp.pprint(J)
J


Jacobiano simbólico:
⎡-I⋅β - μ     -S⋅β    ⎤
⎢                     ⎥
⎣  I⋅β     S⋅β - γ - μ⎦


Matrix([
[-I*beta - mu,             -S*beta],
[      I*beta, S*beta - gamma - mu]])


## 3) Autovalores simbólicos en el **ELE** \((S^*, I^*) = (N, 0)\)


In [3]:
# Jacobiano en ELE
J_ELE = J.subs({S: N_s, I: 0})
eigs_ELE = J_ELE.eigenvals()

print("Jacobiano simbólico en ELE (S=N, I=0):")
sp.pprint(J_ELE)
print("\nAutovalores simbólicos del ELE:")
for val, mult in eigs_ELE.items():
    sp.pprint(sp.Eq(sp.Symbol('lambda'), val))


Jacobiano simbólico en ELE (S=N, I=0):
⎡-μ     -N⋅β    ⎤
⎢               ⎥
⎣0   N⋅β - γ - μ⎦

Autovalores simbólicos del ELE:
λ = N⋅β - γ - μ
λ = -μ


In [4]:
# Verificación simbólica de lambda2
R0_sym = (beta_s*N_s)/(gamma_s + mu_s)
lambda2 = beta_s*N_s - (gamma_s + mu_s)
check = sp.simplify(lambda2 - (gamma_s + mu_s)*(R0_sym - 1))
print("Verificación simbólica de λ2 - (γ+μ)(R0 - 1) == 0  ->", sp.simplify(check) == 0)


Verificación simbólica de λ2 - (γ+μ)(R0 - 1) == 0  -> True



## 4) Equilibrio endémico numérico y Jacobiano numérico



In [5]:
# Equilibrio endémico
S_star = (gamma + mu)/beta
I_star = mu*(N - S_star)/(beta*S_star)
R_star = N - S_star - I_star

print(f"S* = {S_star:.6f}, I* = {I_star:.6f}, R* = {R_star:.6f}")

# Jacobiano numérico en el endémico
J_num = J.subs({
    beta_s: beta, gamma_s: gamma, mu_s: mu, N_s: N, S: S_star, I: I_star
})
print("\nJacobiano numérico en el endémico:")
sp.pprint(J_num)
J_num


S* = 240.000000, I* = 126.666667, R* = 633.333333

Jacobiano numérico en el endémico:
⎡-0.0833333333333333          -0.12        ⎤
⎢                                          ⎥
⎣0.0633333333333333   -1.04083408558608e-17⎦


Matrix([
[-0.0833333333333333,                 -0.12],
[ 0.0633333333333333, -1.04083408558608e-17]])


## 5) Autovalores numéricos del Jacobiano en el equilibrio endémico


In [6]:
import numpy as np

J_np = np.array(J_num.tolist(), dtype=np.float64)
eigvals_end = np.linalg.eigvals(J_np)
print("Autovalores numéricos (endémico):")
print(eigvals_end)

print("\nPartes reales: ", np.real(eigvals_end))


Autovalores numéricos (endémico):
[-0.04166667+0.07657603j -0.04166667-0.07657603j]

Partes reales:  [-0.04166667 -0.04166667]


## 6) Análisis

1) Punto libre de enfermedad (ELE: S=N, I=0)  
- Con nuestros parámetros, R0 ≈ 4.1667 (>1) y el segundo autovalor ≈ 0.38 (positivo).  
- Interpretación: si aparecen pocos infectados, la infección crece y el sistema se aleja de (N,0).  
- Conclusión: ELE inestable.

2) Punto endémico (existe porque R0>1)  
- Valores: S* ≈ 240.0, I* ≈ 126.67, R* ≈ 633.33.  
- Autovalores: ≈ −0.0417 ± 0.0768i (parte real negativa).  
- Interpretación: el sistema regresa al equilibrio con oscilaciones que se apagan; periodo ≈ 82 unidades de tiempo.  
- Conclusión: equilibrio endémico estable (tipo espiral).

3) Efecto de la tasa μ  
- Si μ sube: el crecimiento inicial alrededor del ELE es más lento y el retorno al endémico es más rápido (más amortiguamiento).  
- Si μ baja: ocurre lo contrario.

4) Conexión con la Parte 1  
- Lo visto en las simulaciones se explica así: el sistema sale del ELE (inestable) y termina acercándose al punto endémico con pequeñas oscilaciones hasta estabilizarse.
